In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,848 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,352 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [2]:
# Import packages
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import DataFrame, SparkSession
import time
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [3]:
# Create a SparkSession
spark= SparkSession \
       .builder \
       .appName("Project 4 Group 3 Spark") \
       .getOrCreate()
spark

In [4]:
# Read in data from Github as a DataFrame.
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/AlexFeeney/Project4_Group3/main/Data/healthcare-dataset-stroke-data.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("healthcare-dataset-stroke-data.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-----+------+---+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender|age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+---+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male| 67|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female| 61|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male| 80|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female| 49|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female| 79|           1|            0|         Yes|Self

# Preprocessing

In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

In [6]:
# Convert DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

In [7]:
# Determine the number of unique values in each column.
pandas_df.nunique()

id                   5110
gender                  3
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    3979
bmi                   419
smoking_status          4
stroke                  2
dtype: int64

In [8]:
# Count unique values in column 'gender'
pandas_df['gender'].value_counts()

Female    2994
Male      2115
Other        1
Name: gender, dtype: int64

In [9]:
# Conditionally drop rows where column 'gender' contains the text 'Other'
# pandas_df_clean = pandas_df[~pandas_df['gender'].str.contains('Other')]

# Drop "other" values (case-insensitive and strip whitespaces)
pandas_df_clean = pandas_df[pandas_df['gender'].str.strip().str.lower() != 'other']

pandas_df_clean['gender'].value_counts()

Female    2994
Male      2115
Name: gender, dtype: int64

In [12]:
# Conditionally drop rows where column 'bmi' contains the text 'N/A'
pandas_df_clean = pandas_df[~pandas_df['bmi'].str.contains('N/A')]

pandas_df_clean['gender'].value_counts()

Female    2897
Male      2011
Other        1
Name: gender, dtype: int64

In [13]:
# Drop the non-beneficial 'id' columns
pandas_df_clean = pandas_df_clean.drop(columns = ['id'])
pandas_df_clean.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79,1,0,Yes,Self-employed,Rural,174.12,24,never smoked,1
5,Male,81,0,0,Yes,Private,Urban,186.21,29,formerly smoked,1


In [14]:
# Check data types of columns
print(pandas_df_clean.dtypes)

gender               object
age                  object
hypertension         object
heart_disease        object
ever_married         object
work_type            object
Residence_type       object
avg_glucose_level    object
bmi                  object
smoking_status       object
stroke               object
dtype: object


In [15]:
import numpy as np

# Change data types using astype()
pandas_df_clean['age'] = pandas_df_clean['age'].astype(float)
pandas_df_clean['hypertension'] = pandas_df_clean['hypertension'].astype(np.int64)
pandas_df_clean['avg_glucose_level'] = pandas_df_clean['avg_glucose_level'].astype(float)
pandas_df_clean['bmi'] = pandas_df_clean['bmi'].astype(float)
pandas_df_clean['heart_disease'] = pandas_df_clean['heart_disease'].astype(np.int64)
pandas_df_clean['stroke'] = pandas_df_clean['stroke'].astype(np.int64)


In [16]:
# Check data types of columns
print(pandas_df_clean.dtypes)

gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object


In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
pandas_df_numeric = pd.get_dummies(pandas_df_clean, dtype=int)
pandas_df_numeric.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
2,80.0,0,1,105.92,32.5,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.4,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.0,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5,81.0,0,0,186.21,29.0,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0


In [18]:
# Split pandas_df_numeric into features and target arrays
y = pandas_df_numeric['stroke'].values
X = pandas_df_numeric.drop(columns='stroke').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Checking split data
print (f"X_train: {X_train.shape}")
print (f"X_test: {X_test.shape}")
print (f"y_train: {y_train.shape}")
print (f"y_test: {y_test.shape}")

X_train: (3681, 21)
X_test: (1228, 21)
y_train: (3681,)
y_test: (1228,)


In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train and Evaluate the Model

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_number = len(X_train[0])
hidden_nodes_layer1 = 4
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim = input_features_number))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 88        
                                                                 
 dense_1 (Dense)             (None, 4)                 20        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113 (452.00 Byte)
Trainable params: 113 (452.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Create a callback that saves the model's weights every five epochs
checkpoint_callback = ModelCheckpoint(filepath='model_weights_{epoch:02d}.h5',
                                      save_weights_only=True,
                                      save_freq=5 * len(X_train))  # Save every 5 epochs

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
116/116 [==============================] - 1s 3ms/step - loss: 0.6128 - accuracy: 0.8001
Epoch 2/50
116/116 [==============================] - 0s 2ms/step - loss: 0.4095 - accuracy: 0.9579
Epoch 3/50
116/116 [==============================] - 0s 2ms/step - loss: 0.2641 - accuracy: 0.9579
Epoch 4/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1948 - accuracy: 0.9579
Epoch 5/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1738 - accuracy: 0.9579
Epoch 6/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1658 - accuracy: 0.9579
Epoch 7/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1609 - accuracy: 0.9579
Epoch 8/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1581 - accuracy: 0.9579
Epoch 9/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1559 - accuracy: 0.9579
Epoch 10/50
116/116 [==============================] - 0s 2ms/step - loss: 0.1543 - accuracy: 0.9579

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.1529 - accuracy: 0.9560 - 221ms/epoch - 6ms/step
Loss: 0.1529092937707901, Accuracy: 0.9560260772705078


In [ ]:
# # Export our model to HDF5 file
# filepath = r"keras_model_run0.h5"
# nn.save(filepath, save_format='h5')

# Optimisation
Method: Increasing the number of hidden layers and neurons

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_number = len(X_train[0])
hidden_nodes_layer1 = 4
hidden_nodes_layer2 = 6
hidden_nodes_layer3 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim = input_features_number))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 4)                 88        
                                                                 
 dense_16 (Dense)            (None, 6)                 30        
                                                                 
 dense_17 (Dense)            (None, 8)                 56        
                                                                 
 dense_18 (Dense)            (None, 1)                 9         
                                                                 
Total params: 183 (732.00 Byte)
Trainable params: 183 (732.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Create a callback that saves the model's weights every five epochs
checkpoint_callback = ModelCheckpoint(filepath='model_weights_{epoch:02d}.h5',
                                      save_weights_only=True,
                                      save_freq=5 * len(X_train))  # Save every 5 epochs

In [35]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
116/116 [==============================] - 4s 12ms/step - loss: 0.3697 - accuracy: 0.9579
Epoch 2/50
116/116 [==============================] - 2s 15ms/step - loss: 0.2421 - accuracy: 0.9579
Epoch 3/50
116/116 [==============================] - 1s 8ms/step - loss: 0.1977 - accuracy: 0.9579
Epoch 4/50
116/116 [==============================] - 1s 6ms/step - loss: 0.1792 - accuracy: 0.9579
Epoch 5/50
116/116 [==============================] - 1s 8ms/step - loss: 0.1703 - accuracy: 0.9579
Epoch 6/50
116/116 [==============================] - 1s 8ms/step - loss: 0.1666 - accuracy: 0.9579
Epoch 7/50
116/116 [==============================] - 1s 6ms/step - loss: 0.1637 - accuracy: 0.9579
Epoch 8/50
116/116 [==============================] - 1s 6ms/step - loss: 0.1612 - accuracy: 0.9579
Epoch 9/50
116/116 [==============================] - 1s 6ms/step - loss: 0.1592 - accuracy: 0.9579
Epoch 10/50
116/116 [==============================] - 1s 5ms/step - loss: 0.1574 - accuracy: 0.95

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.1598 - accuracy: 0.9560 - 197ms/epoch - 5ms/step
Loss: 0.15981754660606384, Accuracy: 0.9560260772705078


In [ ]:
# # Export our model to HDF5 file
# filepath = r"keras_model_run1.h5"
# nn.save(filepath, save_format='h5')